In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **STOCKS DATA TRACKING AND STORING FILE**

In [ ]:
# Directories
HOME_DIR = r"/content/drive/MyDrive/Quant Researches - An Algorithmic Trading Platform"
CONFIG_DIR = r"/content/drive/MyDrive/Quant Researches - An Algorithmic Trading Platform/Config"
DATA_DIR = r"/content/drive/MyDrive/Quant Researches - An Algorithmic Trading Platform/Data"
EXECUTION_DIR = r"/content/drive/MyDrive/Quant Researches - An Algorithmic Trading Platform/Execution"
RISK_DIR = r"/content/drive/MyDrive/Quant Researches - An Algorithmic Trading Platform/Risk"
STRATEGIES_DIR = r"/content/drive/MyDrive/Quant Researches - An Algorithmic Trading Platform/Strategies"
STRUCTURES_DIR = r"/content/drive/MyDrive/Quant Researches - An Algorithmic Trading Platform/Structures"
UTILITIES_DIR = r"/content/drive/MyDrive/Quant Researches - An Algorithmic Trading Platform/Utilities"


In [ ]:
import sqlite3
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd

def fetch_and_store_multiple_stocks(ticker_list, start_date, end_date, db_path='stocks.db'):
    # Connect to database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Create the table if it doesn't exist
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS daily_stock_prices (
            ticker TEXT,
            date DATE,
            open REAL,
            high REAL,
            low REAL,
            close REAL,
            volume INTEGER,
            PRIMARY KEY (ticker, date)
        )
    """)
    conn.commit()

    # Download all tickers at once
    try:
        stock_data = yf.download(ticker_list, start=start_date, end=end_date, group_by='ticker', auto_adjust=False, threads=True)
    except Exception as e:
        print(f"Failed to download stock data: {e}")
        conn.close()
        return

    if stock_data.empty:
        print("No data downloaded.")
        conn.close()
        return

    # Handle both single and multiple tickers correctly
    if len(ticker_list) == 1:
        # Single ticker case
        ticker = ticker_list[0]
        rows_to_insert = []
        for index, row in stock_data.iterrows():
            rows_to_insert.append((
                ticker,
                index.strftime("%Y-%m-%d"),
                float(row['Open']),
                float(row['High']),
                float(row['Low']),
                float(row['Close']),
                int(row['Volume'])
            ))
        cursor.executemany("""
            INSERT OR IGNORE INTO daily_stock_prices (ticker, date, open, high, low, close, volume)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        """, rows_to_insert)
        print(f"Inserted {cursor.rowcount} rows for {ticker} into the database.")
    else:
        # Multiple tickers case
        for ticker in ticker_list:
            if ticker not in stock_data.columns.get_level_values(0):
                print(f"No data for {ticker}. Skipping.")
                continue
            rows_to_insert = []
            ticker_data = stock_data[ticker]
            for index, row in ticker_data.iterrows():
                rows_to_insert.append((
                    ticker,
                    index.strftime("%Y-%m-%d"),
                    float(row['Open']),
                    float(row['High']),
                    float(row['Low']),
                    float(row['Close']),
                    int(row['Volume'])
                ))
            cursor.executemany("""
                INSERT OR IGNORE INTO daily_stock_prices (ticker, date, open, high, low, close, volume)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            """, rows_to_insert)
            print(f"Inserted {cursor.rowcount} rows for {ticker} into the database.")

    conn.commit()
    conn.close()
    print("✅ All tickers processed and stored successfully.")

tickers = ['AAPL', 'MSFT', 'TSLA', 'GOOG']
fetch_and_store_multiple_stocks(tickers, '2022-01-01', '2025-12-31')


def retrieve_stock_data(ticker_list, start_date=None, end_date=None, db_path='stocks.db'):
    # Connect to database
    conn = sqlite3.connect(db_path)

    # Handle single ticker as list
    if isinstance(ticker_list, str):
        ticker_list = [ticker_list]

    placeholders = ','.join(['?'] * len(ticker_list))  # Creates (?, ?, ?) dynamically based on number of tickers

    query = f"""
        SELECT ticker, date, open, high, low, close, volume
        FROM daily_stock_prices
        WHERE ticker IN ({placeholders})
    """

    params = list(ticker_list)

    # Add optional date filtering
    if start_date:
        query += " AND date >= ?"
        params.append(start_date)
    if end_date:
        query += " AND date <= ?"
        params.append(end_date)

    query += " ORDER BY ticker, date DESC"

    # Execute the query
    df = pd.read_sql_query(query, conn, params=params, parse_dates=['date'])

    conn.close()

    return df
